In [1]:
import importlib
import funciones_clusters as cl
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
# import re
import pandas as pd
# import seaborn as sns
# import copy
# from mne.viz import plot_evoked_topo
# from mne.channels import find_ch_adjacency, make_1020_channel_selections
# from mne.stats import spatio_temporal_cluster_test, ttest_ind_no_p, spatio_temporal_cluster_1samp_test
# from functools import partial
%matplotlib qt5

In [147]:
importlib.reload(cl)

<module 'funciones_clusters' from 'c:\\Users\\jhquiza\\OneDrive - Universidad de Medellin\\JupyterNotebooks\\IAT\\funciones_clusters.py'>

# Creación de ERPs por tipo de ensayo y sujeto

In [3]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
evocados_co = {}
evocados_in = {}
for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

In [4]:
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'major','legend.fontsize':20, 'lines.linewidth': 2.0 ,'legend.loc': 'best','xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)

# Análisis entre grupos de sujetos

## Creación de diccionarios de ERPs por grupos

In [5]:
df = pd.read_csv('Datos_IAT_comportamentales_personales.csv', index_col='subject')
df.head()

,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,dscore_0,...,DAP,AAP,TPA,TPV,EASPA,ANA,BN,ANC,AGA.1,TAPV
subject,,,,,,,,,,,,,,,,,,,,,
21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,-0.197633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,-0.116918,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,0.473119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,0.214867,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,0.423545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
exbothsides = list(df[df['group']=='ex-both-sides'].index)
exparamilitar = list(df[df['group']=='exparamilitar'].index)
exguerrilla = list(df[df['group']=='exguerrilla'].index)
victim = list(df[df['group']=='victim'].index)
control = list(df[df['group']=='control'].index)

for subject in range(len(exbothsides)):
    exbothsides[subject] = str(exbothsides[subject])
for subject in range(len(victim)):
    victim[subject] = str(victim[subject])
for subject in range(len(exparamilitar)):
    exparamilitar[subject] = str(exparamilitar[subject])
for subject in range(len(exguerrilla)):
    exguerrilla[subject] = str(exguerrilla[subject])
for subject in range(len(control)):
    control[subject] = str(control[subject])

print('count exbothsides: ',len(exbothsides),'\ncount victims: ',len(victim),'\ncount exparamilitaries: ',len(exparamilitar),'\ncount exguerrillas: ',len(exguerrilla), '\ncount controls: ',len(control))

count exbothsides:  5 
count victims:  23 
count exparamilitaries:  31 
count exguerrillas:  22 
count controls:  16


In [7]:
__, __, ambos_co, ambos_in, __, __, __ = cl.matrices_cluster(exbothsides, evocados_co, evocados_in)
__, __, exguerrilleros_co, exguerrilleros_in, __, __, __ = cl.matrices_cluster(exguerrilla, evocados_co, evocados_in)
__, __, exparamilitares_co, exparamilitares_in, __, __, __ = cl.matrices_cluster(exparamilitar, evocados_co, evocados_in)
__, __, victimas_co, victimas_in, __, __, __ = cl.matrices_cluster(victim, evocados_co, evocados_in)
__, __, controles_co, controles_in, tiempos, canales, adjacency = cl.matrices_cluster(control, evocados_co, evocados_in)

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(5, 205, 64)
(5, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(22, 205, 64)
(22, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(31, 205, 64)
(31, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 205, 64)
(23, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delau

In [8]:
controles_dif = controles_co - controles_in
print(np.shape(controles_dif))
exguerrilleros_dif = exguerrilleros_co - exguerrilleros_in
print(np.shape(exguerrilleros_dif))
exparamilitares_dif = exparamilitares_co - exparamilitares_in
print(np.shape(exparamilitares_dif))
victimas_dif = victimas_co - victimas_in
print(np.shape(victimas_dif))
ambos_dif = ambos_co - ambos_in
print(np.shape(ambos_dif))

(16, 205, 64)
(22, 205, 64)
(31, 205, 64)
(23, 205, 64)
(5, 205, 64)


## Análisis univariante masivo todos los grupos menos ambos bandos

In [9]:
# Análisis de ensayos incongruentes exparamilitares, exguerrilleros, víctimas y controles
X = [exguerrilleros_in, exparamilitares_in, victimas_in, controles_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.001320 max=6.680111
Running initial clustering
Using 33 thresholds from 0.20 to 6.60 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [07:20<00:00,    2.27it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [10]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_in = cl.clusters_color(df=df_p_values)

In [61]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>25]

C1     66.0
FCZ    64.0
F1     62.0
FC6    51.0
C6     51.0
F3     48.0
T8     46.0
FC4    46.0
CZ     46.0
FZ     45.0
FC1    44.0
C3     44.0
FC3    43.0
FC2    40.0
FT8    37.0
P10    36.0
AF3    36.0
F8     36.0
CP1    33.0
P1     33.0
F7     32.0
T7     31.0
FP1    30.0
CP3    29.0
FC5    29.0
FT7    28.0
F5     28.0
CP6    26.0
F2     26.0
dtype: float64

In [29]:
cl.time_intervals(df=df_electrodos, electrodos=['C1', 'FCZ', 'F1'])

,cuenta,t_inicial,t_final,delta(ms)
204,60,0.566406,0.796875,230.46875


In [50]:
cl.time_intervals(df=df_electrodos, electrodos=['C1', 'FCZ', 'F1','F3','FC3','FC1', 'C3'])

,cuenta,t_inicial,t_final,delta(ms)
204,38,0.652344,0.796875,144.53125


In [55]:
cl.time_intervals(df=df_electrodos, electrodos=['T8','C6', 'FC6', 'FC4'])

,cuenta,t_inicial,t_final,delta(ms)
204,32,0.675781,0.796875,121.09375


In [64]:
cl.time_intervals(df=df_electrodos, electrodos=['T7'])

,cuenta,t_inicial,t_final,delta(ms)
173,26,0.578125,0.675781,97.65625


In [67]:
cl.time_intervals(df=df_electrodos, electrodos=['P10'])

,cuenta,t_inicial,t_final,delta(ms)
181,18,0.640625,0.707031,66.40625


In [68]:
cl.time_intervals(df=df_electrodos, electrodos=['P1'])

,cuenta,t_inicial,t_final,delta(ms)
180,12,0.660156,0.703125,42.96875
204,21,0.718750,0.796875,78.12500


In [69]:
# Análisis de ensayos congruentes exparamilitares, exguerrilleros, víctimas y controles
X = [exguerrilleros_co, exparamilitares_co, victimas_co, controles_co]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.001942 max=6.375320
Running initial clustering
Using 31 thresholds from 0.20 to 6.20 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [07:27<00:00,    2.23it/s]

Computing cluster p-values
Done.


0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [70]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_co = cl.clusters_color(df=df_p_values)

In [71]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>25]

FZ     72.0
FC1    72.0
F1     69.0
F3     63.0
FCZ    61.0
AF3    58.0
FC3    55.0
FP1    51.0
F2     50.0
O2     50.0
FC5    47.0
CZ     47.0
PO8    37.0
C1     34.0
POZ    32.0
C3     29.0
OZ     27.0
IZ     26.0
dtype: float64

In [102]:
cl.time_intervals(df=df_electrodos, electrodos=['F2', 'FZ'])

,cuenta,t_inicial,t_final,delta(ms)
56,14,0.167969,0.218750,50.78125
119,19,0.394531,0.464844,70.31250


In [90]:
cl.time_intervals(df=df_electrodos, electrodos=['F5', 'F3', 'F1', 'FZ','FC1', 'FC3', 'FC5','AF3'])

,cuenta,t_inicial,t_final,delta(ms)
81,14,0.265625,0.316406,50.78125


In [94]:
cl.time_intervals(df=df_electrodos, electrodos=['F1', 'F3', 'FZ'])

,cuenta,t_inicial,t_final,delta(ms)
83,20,0.250000,0.324219,74.21875
119,34,0.335938,0.464844,128.90625


In [107]:
cl.time_intervals(df=df_electrodos, electrodos=['F2', 'FZ', 'FC2'])

,cuenta,t_inicial,t_final,delta(ms)
117,17,0.394531,0.457031,62.5


In [109]:
# Análisis de diferencias ensayos congruentes e incongruentes exparamilitares, exguerrilleros, víctimas y controles
X = [exguerrilleros_dif, exparamilitares_dif, victimas_dif, controles_dif]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000366 max=8.618625
Running initial clustering
Using 43 thresholds from 0.20 to 8.60 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [07:42<00:00,    2.16it/s]

Computing cluster p-values


Done.
5 points selected by TFCE ...


In [110]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_dif = cl.clusters_color(df=df_p_values)

In [149]:
figura_dif = cl.clusters_color(df=df_p_values)

In [111]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>25]

F3     118.0
FC3    103.0
FC1    103.0
F1     101.0
CZ     101.0
FCZ     97.0
C3      97.0
CP3     95.0
C1      95.0
AF3     84.0
CP1     64.0
FZ      63.0
F5      60.0
P1      58.0
FC2     51.0
FC5     48.0
P3      45.0
FP1     43.0
F2      41.0
C5      33.0
CPZ     33.0
AFZ     29.0
dtype: float64

In [125]:
cl.time_intervals(df=df_electrodos, electrodos=['F3', 'FC3', 'FC1', 'F1', 'C1', 'CZ'])

,cuenta,t_inicial,t_final,delta(ms)
56,14,0.167969,0.218750,50.78125
89,16,0.289062,0.347656,58.59375
114,13,0.398438,0.445312,46.87500
145,12,0.523438,0.566406,42.96875


In [134]:
cl.time_intervals(df=df_electrodos, electrodos=['FC3', 'FC1', 'F1', 'F3', 'AF3'])

,cuenta,t_inicial,t_final,delta(ms)
91,31,0.238281,0.355469,117.1875
114,13,0.398438,0.445312,46.8750


In [136]:
cl.time_intervals(df=df_electrodos, electrodos=['FC3','F3'])

,cuenta,t_inicial,t_final,delta(ms)
57,17,0.160156,0.222656,62.50000
95,35,0.238281,0.371094,132.81250
125,24,0.398438,0.488281,89.84375
145,18,0.500000,0.566406,66.40625


In [144]:
cl.time_intervals(df=df_electrodos, electrodos=['C1', 'C3', 'CZ'])

,cuenta,t_inicial,t_final,delta(ms)
89,16,0.289062,0.347656,58.59375
115,14,0.398438,0.449219,50.78125
155,32,0.484375,0.605469,121.09375


In [145]:
cl.time_intervals(df=df_electrodos, electrodos=['P1'])

,cuenta,t_inicial,t_final,delta(ms)
143,20,0.484375,0.558594,74.21875


In [147]:
cl.time_intervals(df=df_electrodos, electrodos=['P3'])

,cuenta,t_inicial,t_final,delta(ms)
139,17,0.480469,0.542969,62.5


In [148]:
cl.time_intervals(df=df_electrodos, electrodos=['F2'])

,cuenta,t_inicial,t_final,delta(ms)
85,19,0.261719,0.332031,70.3125


## Gráficas de ERP y extracción de medidas de clústeres

In [151]:
# Gráfica ERP clúster 1
canales_1 = ['F1', 'F3', 'FC1', 'FC3', 'C1', 'CZ']
t_init_1 = 0.168
duration_1 = 0.0508
fig1 = cl.graficos_grupos_control(canales=canales_1, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_1, duration=duration_1, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [152]:
# Gráfica ERP clúster 2
canales_2 = ['F1', 'F3', 'FC1', 'FC3', 'AF3']
t_init_2 = 0.2383
duration_2 = 0.1172
fig1 = cl.graficos_grupos_control(canales=canales_2, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_2, duration=duration_2, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [153]:
# Gráfica ERP clúster 3
canales_3 = ['F1', 'F3', 'FZ']
t_init_3 = 0.3359
duration_3 = 0.1289
fig1 = cl.graficos_grupos_control(canales=canales_3, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_3, duration=duration_3, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [154]:
# Gráfica ERP clúster 4
canales_4 = ['C1', 'C3', 'CZ']
t_init_4 = 0.4844
duration_4 = 0.1211
fig1 = cl.graficos_grupos_control(canales=canales_4, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_4, duration=duration_4, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [155]:
# Gráfica ERP clúster 5
canales_5 = ['C1', 'FCZ', 'F1']
t_init_5 = 0.5664
duration_5 = 0.2305
fig1 = cl.graficos_grupos_control(canales=canales_5, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_5, duration=duration_5, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [158]:
# Gráfica ERP clúster 6
canales_6 = ['T8', 'C6', 'FC6', 'FC4']
t_init_6 = 0.6758
duration_6 = 0.1211
fig1 = cl.graficos_grupos_control(canales=canales_6, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, 
                          exparamilitar=exparamilitar, exguerrilla=exguerrilla, victim=victim, control=control, 
                          t_init=t_init_6, duration=duration_6, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [159]:
cluster_1 = ', '.join(canales_1)
cluster_2 = ', '.join(canales_2)
cluster_3 = ', '.join(canales_3)
cluster_4 = ', '.join(canales_4)
cluster_5 = ', '.join(canales_5)
cluster_6 = ', '.join(canales_6)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1], [cluster_2, t_init_2, t_init_2+duration_2], 
                               [cluster_3, t_init_3, t_init_3+duration_3], [cluster_4, t_init_4, t_init_4+duration_4],
                               [cluster_5, t_init_5, t_init_5+duration_5], [cluster_6, t_init_6, t_init_6+duration_6]],
                               columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_grupos(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, 
                                    exbothsides=exbothsides, exparamilitar=exparamilitar,
                                    exguerrilla=exguerrilla, victim=victim, control=control)
df_medidas.head()

c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de M

,subject,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2,mean_co_3,mean_in_3,dif_co_in_3,mean_co_4,mean_in_4,dif_co_in_4,mean_co_5,mean_in_5,dif_co_in_5
0,23001,-0.127033,-0.768203,0.641170,1.247550,0.272750,0.974800,-0.267636,-0.030627,-0.237008,-1.279037,-1.072705,-0.206333,-0.610755,-1.285424,0.674670,1.092875,1.168829,-0.075954
1,23003,0.835522,1.305722,-0.470200,1.243344,1.257732,-0.014388,0.871470,0.824179,0.047291,0.663652,0.406921,0.256731,0.943438,0.001939,0.941500,-0.124950,-0.751880,0.626930
2,23008,-0.079642,-1.054652,0.975009,0.097159,-0.304574,0.401733,-0.525595,-0.218977,-0.306617,-0.701117,-1.087294,0.386177,-0.347085,-0.456085,0.109000,0.292954,0.568072,-0.275119
3,23022,2.557134,0.804571,1.752564,3.584544,2.491719,1.092825,1.882318,1.524185,0.358134,1.050253,-0.671937,1.722190,0.396830,-0.894592,1.291422,0.317912,1.643076,-1.325164
4,23025,1.829664,-1.307322,3.136986,1.567784,0.023702,1.544082,0.625249,-0.375560,1.000809,0.133488,-0.882515,1.016003,1.409972,0.503315,0.906657,-0.109544,0.524437,-0.633981


In [160]:
df_medidas['subject']= df_medidas['subject'].astype('int64')
df_medidas.set_index('subject', inplace=True)
medidas = pd.merge(df, df_medidas, left_index=True, right_index=True, how='inner')
medidas.head()

,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,dscore_0,...,dif_co_in_2,mean_co_3,mean_in_3,dif_co_in_3,mean_co_4,mean_in_4,dif_co_in_4,mean_co_5,mean_in_5,dif_co_in_5
subject,,,,,,,,,,,,,,,,,,,,,
21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,-0.197633,...,-1.872368,0.465257,1.378642,-0.913385,-1.589684,1.365510,-2.955193,0.603834,0.700703,-0.096869
21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,-0.116918,...,0.810757,2.221917,1.724073,0.497843,1.487915,1.256293,0.231622,1.112225,1.055844,0.056382
21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,0.473119,...,2.624299,-1.697306,-1.555080,-0.142226,-0.233696,-1.119933,0.886236,2.452845,3.443613,-0.990768
21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,0.214867,...,-1.162934,-2.017112,0.369584,-2.386696,-1.742834,-0.065690,-1.677144,-1.968273,0.790077,-2.758350
21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,0.423545,...,-0.992374,0.808308,1.676205,-0.867897,1.631783,2.465775,-0.833992,0.198569,1.374529,-1.175960


In [161]:
medidas_clusters_grupos_no_exb = medidas[['type', 'exposure_level', 'group', 'victim_self','age', 'scholarship', 'gender', 
                                          'laterality', 'dscore_4', 'modo_2', 'mean_co_0', 'mean_in_0', 'dif_co_in_0', 
                                          'mean_co_1', 'mean_in_1', 'dif_co_in_1', 'mean_co_2', 'mean_in_2', 'dif_co_in_2', 
                                          'mean_co_3', 'mean_in_3', 'dif_co_in_3', 'mean_co_4', 'mean_in_4', 'dif_co_in_4', 
                                          'mean_co_5', 'mean_in_5', 'dif_co_in_5']]

medidas_clusters_grupos_no_exb = medidas_clusters_grupos_no_exb[medidas_clusters_grupos_no_exb['group']!='ex-both-sides']
medidas_clusters_grupos_no_exb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 21100 to 24101
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            92 non-null     object 
 1   exposure_level  92 non-null     object 
 2   group           92 non-null     object 
 3   victim_self     89 non-null     object 
 4   age             92 non-null     int64  
 5   scholarship     92 non-null     int64  
 6   gender          92 non-null     object 
 7   laterality      92 non-null     object 
 8   dscore_4        91 non-null     float64
 9   modo_2          91 non-null     object 
 10  mean_co_0       92 non-null     float64
 11  mean_in_0       92 non-null     float64
 12  dif_co_in_0     92 non-null     float64
 13  mean_co_1       92 non-null     float64
 14  mean_in_1       92 non-null     float64
 15  dif_co_in_1     92 non-null     float64
 16  mean_co_2       92 non-null     float64
 17  mean_in_2       92 non-null   

In [162]:
medidas_clusters_grupos_no_exb.to_csv('medidas_clusters_grupos_no_exb.csv')

## Creación de diccionarios de ERPs exparamilitares, exguerrilleros, víctimas

In [19]:
# Solo se analizarán los ERP a partir de 350 ms
__, __, ambos_co, ambos_in, __, __, __ = cl.matrices_cluster(exbothsides, evocados_co, evocados_in, tmin=0.35)
__, __, exguerrilleros_co, exguerrilleros_in, __, __, __ = cl.matrices_cluster(exguerrilla, evocados_co, evocados_in, tmin=0.35)
__, __, exparamilitares_co, exparamilitares_in, __, __, __ = cl.matrices_cluster(exparamilitar, evocados_co, evocados_in, tmin=0.35)
__, __, victimas_co, victimas_in, __, __, __ = cl.matrices_cluster(victim, evocados_co, evocados_in, tmin=0.35)
__, __, controles_co, controles_in, tiempos, canales, adjacency = cl.matrices_cluster(control, evocados_co, evocados_in, tmin=0.35)

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(5, 115, 64)
(5, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(22, 115, 64)
(22, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(31, 115, 64)
(31, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 115, 64)
(23, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delau

In [20]:
controles_dif = controles_co - controles_in
print(np.shape(controles_dif))
exguerrilleros_dif = exguerrilleros_co - exguerrilleros_in
print(np.shape(exguerrilleros_dif))
exparamilitares_dif = exparamilitares_co - exparamilitares_in
print(np.shape(exparamilitares_dif))
victimas_dif = victimas_co - victimas_in
print(np.shape(victimas_dif))
ambos_dif = ambos_co - ambos_in
print(np.shape(ambos_dif))

(16, 115, 64)
(22, 115, 64)
(31, 115, 64)
(23, 115, 64)
(5, 115, 64)


## Análisis univariantes masivos exparamilitares, exguerrilleros, víctimas

In [130]:
# Análisis de ensayos incongruentes exparamilitares, exguerrilleros y víctimas
X = [exguerrilleros_in, exparamilitares_in, victimas_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000223 max=8.232128
Running initial clustering
Using 41 thresholds from 0.20 to 8.20 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [05:04<00:00,    3.28it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [131]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_in = cl.clusters_color(df=df_p_values)

In [132]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

FC6    56.0
C6     48.0
FC4    48.0
P1     35.0
F8     26.0
T8     26.0
CP1    26.0
P3     25.0
CP6    24.0
PZ     23.0
F6     22.0
FP1    21.0
FCZ    19.0
POZ    18.0
FC2    18.0
FZ     16.0
F4     15.0
C1     13.0
FT8    13.0
dtype: float64

In [133]:
cl.time_intervals(df=df_electrodos, electrodos=['FC6', 'C6', 'FC4'])

,cuenta,t_inicial,t_final,delta(ms)
114,32,0.675781,0.796875,121.09375


In [134]:
cl.time_intervals(df=df_electrodos, electrodos=['P1', 'P3'])

,cuenta,t_inicial,t_final,delta(ms)
114,22,0.714844,0.796875,82.03125


In [135]:
cl.time_intervals(df=df_electrodos, electrodos=['FC6', 'C6', 'FC4', 'F6', 'F8'])

,cuenta,t_inicial,t_final,delta(ms)
114,22,0.714844,0.796875,82.03125


In [136]:
# Análisis de ensayos congruentes exparamilitares, exguerrilleros y víctimas
X = [exguerrilleros_co, exparamilitares_co, victimas_co]
f_obs, p_values, umbral = cl.analisis_cluster_f(X=X, adjacency=adjacency)

stat_fun(H1): min=0.000035 max=7.419543
Running initial clustering
Using 37 thresholds from 0.20 to 7.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [05:44<00:00,    2.90it/s]

Computing cluster p-values
Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [137]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_co = cl.clusters_color(df=df_p_values)

In [138]:
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

F1     25.0
F3     25.0
FC1    23.0
FZ     20.0
F2     13.0
dtype: float64

In [142]:
cl.time_intervals(df=df_electrodos, electrodos=['F1', 'F3'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
24,25,0.351562,0.445312,93.75


In [143]:
# Análisis de diferencias entre ensayos congruentes e incongruentes
X = [exguerrilleros_dif, exparamilitares_dif, victimas_dif]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000166 max=8.054419
Running initial clustering
Using 40 thresholds from 0.20 to 8.00 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [05:14<00:00,    3.17it/s]

Computing cluster p-values
Done.


0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [144]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura_dif = cl.clusters_color(df=df_p_values)

In [145]:
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

Series([], dtype: float64)

## Gráficas de ERP y extracción de medidas por clústeres

In [180]:
# Gráfica ERP clúster 1
canales_1 = ['F1', 'F3']
t_init_1 = 0.35
duration_1 = 0.1
fig1 = cl.graficos_grupos(canales=canales_1, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, exparamilitar=exparamilitar, 
                          exguerrilla=exguerrilla, victim=victim, control=control, t_init=t_init_1, duration=duration_1, tmin=0.0, tmax=0.796875, pos_legend='lower left')

In [181]:
# ERP clúster 2
canales_2 = ['FC6', 'C6', 'FC4', 'F6', 'F8']
t_init_2 = 0.71
duration_2 = 0.09
fig2 = cl.graficos_grupos(canales=canales_2, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, exparamilitar=exparamilitar, 
                          exguerrilla=exguerrilla, victim=victim, control=control, t_init=t_init_2, duration=duration_2, tmin=0.0, tmax=0.796875, pos_legend='lower left')

In [183]:
# ERP clúster 3
canales_3 = ['P1', 'P3']
t_init_3 = 0.71
duration_3 = 0.09
fig3 = cl.graficos_grupos(canales=canales_3, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, exparamilitar=exparamilitar, 
                          exguerrilla=exguerrilla, victim=victim, control=control, t_init=t_init_3, duration=duration_3, tmin=0.0, tmax=0.796875, pos_legend='upper left')

In [202]:
cluster_1 = ', '.join(canales_1)
cluster_2 = ', '.join(canales_2)
cluster_3 = ', '.join(canales_3)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1], [cluster_2, t_init_2, t_init_2+duration_2], [cluster_3, t_init_3, t_init_3+duration_3]],
                         columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_grupos(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, exparamilitar=exparamilitar, 
                                 exguerrilla=exguerrilla, victim=victim, control=control)
df_medidas.head()

c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de M

,subject,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2
0,23001,-0.330097,-0.171776,-0.158321,1.555896,1.436985,0.118911,-1.358550,-2.375457,1.016907
1,23003,1.018205,0.910234,0.107972,0.025174,-0.603625,0.628800,0.697434,-1.330713,2.028147
2,23008,-0.676873,-0.278449,-0.398424,0.497347,0.482012,0.015335,-1.095792,-1.020830,-0.074963
3,23022,1.993540,1.649387,0.344153,0.668477,2.036175,-1.367698,0.843168,0.948915,-0.105747
4,23025,0.554653,-0.229917,0.784570,-0.679233,0.751010,-1.430242,1.252069,-1.328047,2.580117


In [203]:
df_medidas['subject']= df_medidas['subject'].astype('int64')
df_medidas.set_index('subject', inplace=True)
medidas = pd.merge(df, df_medidas, left_index=True, right_index=True, how='inner')
medidas.head()

In [207]:
medidas_clusters_exp_exg_vic_350 = medidas[['type','group', 'age', 'scholarship', 'gender', 'laterality', 'dscore_4', 'modo_2', 'mean_co_0', 'mean_in_0',
                                             'dif_co_in_0', 'mean_co_1', 'mean_in_1', 'dif_co_in_1', 'mean_co_2', 'mean_in_2', 'dif_co_in_2']]

medidas_clusters_exp_exg_vic_350 = medidas_clusters_exp_exg_vic_350[(medidas_clusters_exp_exg_vic_350['group']=='exparamilitar') |
                                                                    (medidas_clusters_exp_exg_vic_350['group']=='exguerrilla') | (medidas_clusters_exp_exg_vic_350['group']=='victim')]
medidas_clusters_exp_exg_vic_350.info()

,type,group,age,scholarship,gender,laterality,dscore_4,modo_2,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2
subject,,,,,,,,,,,,,,,,,
21100,excombatant,exguerrilla,19,11,F,D,0.319044,0.940969009,-0.902704,0.741797,-1.644501,0.641788,1.006168,-0.364379,0.325075,0.092656,0.232419
21101,excombatant,exparamilitar,46,11,M,D,-0.070836,0.006873246,1.606104,0.765813,0.840291,-0.019495,0.937055,-0.956550,2.967996,1.530142,1.437853
21102,excombatant,exparamilitar,31,11,M,D,0.488057,#N/D,4.456430,1.014983,3.441448,0.638641,0.824989,-0.186348,-0.717015,0.682201,-1.399216
21103,excombatant,exparamilitar,42,18,M,D,0.120374,0.370198186,-0.826131,0.615661,-1.441792,-2.617450,0.307316,-2.924766,-1.451092,0.435904,-1.886996
21104,excombatant,exparamilitar,38,11,M,D,0.243338,0.42363553,-0.604626,1.269987,-1.874613,-0.222800,1.061643,-1.284444,1.212636,2.176844,-0.964208


In [212]:
medidas_clusters_exp_exg_vic_350.to_csv('medidas_clusters_exp_exg_vic_350.csv')

# Análisis excombatientes víctimas

## Creación de diccionarios de ERPs por grupos

In [4]:
data = pd.read_csv('Datos_IAT_comportamentales_personales.csv', index_col='subject')
df = data[data['group']!='ex-both-sides']

excombatant = list(df[df['type']=='excombatant'].index)
victim = list(df[df['type']=='victim'].index)
control = list(df[df['type']=='control'].index)

for subject in range(len(victim)):
    victim[subject] = str(victim[subject])
for subject in range(len(excombatant)):
    excombatant[subject] = str(excombatant[subject])
for subject in range(len(control)):
    control[subject] = str(control[subject])

print('count victims: ',len(victim),'\ncount excombatants: ',len(excombatant), '\ncount controls: ',len(control))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 21100 to 24101
Columns: 164 entries, type to TAPV
dtypes: float64(147), int64(2), object(15)
memory usage: 118.6+ KB


In [6]:
__, __, excombatientes_co, excombatientes_in, __, __, __ = cl.matrices_cluster(excombatant, evocados_co, evocados_in, tmin=0.35)
__, __, victimas_co, victimas_in, __, __, __ = cl.matrices_cluster(victim, evocados_co, evocados_in, tmin=0.35)
__, __, controles_co, controles_in, tiempos, canales, adjacency = cl.matrices_cluster(control, evocados_co, evocados_in, tmin=0.35)

controles_dif = controles_co - controles_in
print(np.shape(controles_dif))
excombatientes_dif = excombatientes_co - excombatientes_in
print(np.shape(excombatientes_dif))
victimas_dif = victimas_co - victimas_in
print(np.shape(victimas_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(53, 115, 64)
(53, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 115, 64)
(23, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(16, 115, 64)
(16, 115, 64)


## Análisis univariantes masivos

In [19]:
# Análisis de ensayos incongruentes
X = [excombatientes_in, victimas_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=16.660490
Running initial clustering
Using 83 thresholds from 0.20 to 16.60 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [06:29<00:00,    2.56it/s]

Computing cluster p-values
Done.


0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [40]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_in = cl.clusters_color(df=df_p_values)

In [10]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

FP1    50.0
FCZ    39.0
P1     38.0
CP1    36.0
FC2    32.0
CPZ    32.0
P3     32.0
PZ     30.0
FPZ    29.0
C1     28.0
CP3    28.0
T7     27.0
POZ    27.0
TP7    22.0
CZ     19.0
PO3    18.0
AFZ    15.0
P9     14.0
FP2    14.0
CP2    13.0
FZ     13.0
dtype: float64

In [11]:
cl.time_intervals(df=df_electrodos, electrodos=['FP1'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
91,42,0.546875,0.707031,160.15625


In [12]:
cl.time_intervals(df=df_electrodos, electrodos=['FCZ', 'FC2'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
71,17,0.566406,0.628906,62.5


In [18]:
cl.time_intervals(df=df_electrodos, electrodos=['C1'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
71,15,0.574219,0.628906,54.6875


In [23]:
cl.time_intervals(df=df_electrodos, electrodos=['T7'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
83,27,0.574219,0.675781,101.5625


In [13]:
cl.time_intervals(df=df_electrodos, electrodos=['P1', 'CP1', 'P3'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
114,22,0.714844,0.796875,82.03125


In [19]:
cl.time_intervals(df=df_electrodos, electrodos=['PZ', 'CPZ'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
113,16,0.734375,0.792969,58.59375


In [23]:
# Análisis de ensayos congruentes
X = [excombatientes_co, victimas_co]
__, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=14.461012
Running initial clustering
Using 72 thresholds from 0.20 to 14.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [06:24<00:00,    2.60it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [24]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_co = cl.clusters_color(df=df_p_values)

In [10]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

F3     35.0
F2     35.0
F1     33.0
FZ     33.0
F4     32.0
AFZ    32.0
AF3    30.0
FC3    29.0
FC1    28.0
FCZ    27.0
FC4    25.0
FC2    21.0
CZ     18.0
C1     17.0
C3     15.0
dtype: float64

In [14]:
cl.time_intervals(df=df_electrodos, electrodos=['F3', 'F1', 'FZ', 'F2', 'AF3', 'FC3'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
28,29,0.351562,0.460938,109.375


In [25]:
# Análisis de diferencias entre ensayos
X = [excombatientes_dif, victimas_dif]
__, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=15.375576
Running initial clustering
Using 76 thresholds from 0.20 to 15.20 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [06:53<00:00,    2.42it/s]

Computing cluster p-values
Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [26]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_dif = cl.clusters_color(df=df_p_values)

In [17]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

F3     23.0
AF3    20.0
F1     15.0
dtype: float64

In [18]:
cl.time_intervals(df=df_electrodos, electrodos=['F3', 'AF3'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
24,16,0.386719,0.445312,58.59375


In [293]:
cl.time_intervals(df=df_electrodos, electrodos=['AFZ', 'AF3', 'F3'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
55,13,0.519531,0.566406,46.875


## Gráficos de ERP y extracción de medidas de clústeres

In [27]:
# Gráfica ERP clúster 1
canales_1 = ['F1','F3', 'FZ', 'F2', 'AF3', 'FC3']
t_init_1 = 0.352
duration_1 = 0.109
fig1 = cl.graficos_exc_vic(canales=canales_1, evocados_co=evocados_co, evocados_in=evocados_in, excombatant=excombatant, 
                           victim=victim, control=control, t_init=t_init_1, duration=duration_1, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [29]:
# Gráfica ERP clúster 2
canales_2 = ['FCZ', 'FC2']
t_init_2 = 0.566
duration_2 = 0.063
fig1 = cl.graficos_exc_vic(canales=canales_2, evocados_co=evocados_co, evocados_in=evocados_in, excombatant=excombatant, 
                           victim=victim, control=control, t_init=t_init_2, duration=duration_2, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [30]:
# Gráfica ERP clúster 3
canales_3 = ['P1', 'P3', 'CP1']
t_init_3 = 0.715
duration_3 = 0.082
fig1 = cl.graficos_exc_vic(canales=canales_3, evocados_co=evocados_co, evocados_in=evocados_in, excombatant=excombatant, 
                           victim=victim, control=control, t_init=t_init_3, duration=duration_3, tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [31]:
cluster_1 = ', '.join(canales_1)
cluster_2 = ', '.join(canales_2)
cluster_3 = ', '.join(canales_3)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1], [cluster_2, t_init_2, t_init_2+duration_2], [cluster_3, t_init_3, t_init_3+duration_3]], 
                         columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_exc_vic(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, excombatant=excombatant, victim=victim, control=control)
df_medidas.head()

c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de M

,subject,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2
0,24002,0.747859,0.665156,0.082703,0.190420,1.234148,-1.043729,2.537447,2.232139,0.305308
1,24003,-1.666677,-1.203568,-0.463109,1.383888,1.251016,0.132872,-0.002740,0.447733,-0.450473
2,24004,-1.331218,-2.234944,0.903726,2.463312,1.541329,0.921983,0.633772,0.628210,0.005562
3,24005,-0.475310,-0.276043,-0.199267,-0.029602,0.338425,-0.368027,0.668102,-0.148662,0.816765
4,24013,-0.841754,-0.660228,-0.181526,2.213723,1.320195,0.893528,0.792387,1.008957,-0.216570


In [32]:
df_medidas['subject']= df_medidas['subject'].astype('int64')
df_medidas.set_index('subject', inplace=True)
medidas = pd.merge(df, df_medidas, left_index=True, right_index=True, how='inner')
medidas.head()

,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,dscore_0,...,TAPV,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2
subject,,,,,,,,,,,,,,,,,,,,,
21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,-0.197633,...,0.0,-1.013927,0.818593,-1.832521,-1.195966,2.136550,-3.332517,0.120540,0.367515,-0.246975
21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,-0.116918,...,1.0,1.641577,0.723454,0.918123,2.193162,2.259026,-0.065865,2.641224,1.497337,1.143886
21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,0.473119,...,0.0,1.570159,-0.521422,2.091581,-3.464134,-2.932099,-0.532034,-1.973695,-0.779295,-1.194400
21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,0.214867,...,0.0,-0.673351,0.883599,-1.556951,-2.524528,-0.308240,-2.216288,-1.534830,0.341205,-1.876035
21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,0.423545,...,0.0,0.415129,1.391235,-0.976107,2.476594,2.561377,-0.084783,1.457886,1.622816,-0.164931


In [33]:
medidas_clusters_exc_vic_no_exb_350 = medidas[['type','group', 'age', 'scholarship', 'gender', 'laterality', 'dscore_4', 'modo_2', 'mean_co_0', 'mean_in_0',
                                             'dif_co_in_0', 'mean_co_1', 'mean_in_1', 'dif_co_in_1', 'mean_co_2', 'mean_in_2', 'dif_co_in_2']]

medidas_clusters_exc_vic_no_exb_350 = medidas_clusters_exc_vic_no_exb_350[(medidas_clusters_exc_vic_no_exb_350['type']=='excombatant') | (medidas_clusters_exc_vic_no_exb_350['type']=='victim')]
medidas_clusters_exc_vic_no_exb_350.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 21100 to 24101
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         76 non-null     object 
 1   group        76 non-null     object 
 2   age          76 non-null     int64  
 3   scholarship  76 non-null     int64  
 4   gender       76 non-null     object 
 5   laterality   76 non-null     object 
 6   dscore_4     76 non-null     float64
 7   modo_2       76 non-null     object 
 8   mean_co_0    76 non-null     float64
 9   mean_in_0    76 non-null     float64
 10  dif_co_in_0  76 non-null     float64
 11  mean_co_1    76 non-null     float64
 12  mean_in_1    76 non-null     float64
 13  dif_co_in_1  76 non-null     float64
 14  mean_co_2    76 non-null     float64
 15  mean_in_2    76 non-null     float64
 16  dif_co_in_2  76 non-null     float64
dtypes: float64(10), int64(2), object(5)
memory usage: 10.7+ KB


In [34]:
medidas_clusters_exc_vic_no_exb_350.to_csv('medidas_clusters_exc_vic_no_exb_350.csv')

# Análisis comparativo entre sí-victimas y no-víctimas

## Creación de diccionarios de ERPs por grupos

In [156]:
data = pd.read_csv('Datos_IAT_comportamentales_personales.csv', index_col='subject')
# Excluyo excombatientes de ambos bandos y controles
df = data[(data['group']!='ex-both-sides')&(data['group']!='control')]

victim = list(df[df['victim_self']=='yes'].index)
no_victim = list(df[df['victim_self']=='no'].index)

for subject in range(len(victim)):
    victim[subject] = str(victim[subject])
for subject in range(len(no_victim)):
    no_victim[subject] = str(no_victim[subject])

print('count victims: ',len(victim),'\ncount no-victim: ',len(no_victim))

count victims:  56 
count no-victim:  18


In [157]:
__, __, victimas_co, victimas_in, __, __, __ = cl.matrices_cluster(victim, evocados_co, evocados_in, tmin=0.35)
__, __, no_victimas_co, no_victimas_in, tiempos, canales, adjacency = cl.matrices_cluster(no_victim, evocados_co, evocados_in, tmin=0.35)

no_victimas_dif = no_victimas_co - no_victimas_in
print(np.shape(no_victimas_dif))
victimas_dif = victimas_co - victimas_in
print(np.shape(victimas_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(56, 115, 64)
(56, 115, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(18, 115, 64)
(18, 115, 64)
(18, 115, 64)
(56, 115, 64)


## Análisis univariantes masivos

In [39]:
# Análisis de ensayos incongruentes
X = [no_victimas_in, victimas_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=10.425036
Running initial clustering
Using 52 thresholds from 0.20 to 10.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [06:23<00:00,    2.60it/s]

Computing cluster p-values
Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [41]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_in = cl.clusters_color(df=df_p_values)

In [42]:
# Análisis de ensayos incongruentes
X = [no_victimas_co, victimas_co]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=8.953333
Running initial clustering
Using 44 thresholds from 0.20 to 8.80 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [06:36<00:00,    2.52it/s]

Computing cluster p-values
Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [43]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_co = cl.clusters_color(df=df_p_values)

In [44]:
# Análisis de ensayos incongruentes
X = [no_victimas_dif, victimas_dif]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000000 max=15.571862
Running initial clustering
Using 77 thresholds from 0.20 to 15.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 7360 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [07:10<00:00,    2.32it/s]

Computing cluster p-values
Done.


0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [45]:
df_p_values = cl.p_values_table(p_values=p_values, tiempos=tiempos, canales=canales)
figura_dif = cl.clusters_color(df=df_p_values)

In [46]:
# df_clusteres = cl.lista_clusteres(df_p_values)
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>12]

C2     45.0
CP2    44.0
CP4    39.0
C4     34.0
FC2    32.0
FC4    25.0
P2     23.0
FC1    20.0
CP1    17.0
FC6    15.0
CZ     15.0
C1     15.0
FC3    14.0
F1     14.0
F6     14.0
F4     13.0
P4     13.0
PO4    13.0
dtype: float64

In [54]:
cl.time_intervals(df=df_electrodos, electrodos=['C2', 'CP2', 'CP4'], min_duration=40)

,cuenta,t_inicial,t_final,delta(ms)
66,34,0.480469,0.609375,128.90625


## Gráficos de ERP y extracción de medidas de clústeres

In [158]:
# Gráfica ERP clúster 1
canales_1 = ['C2', 'CP2', 'CP4']
t_init_1 = 0.480
duration_1 = 0.129
fig1 = cl.graficos_self_vic(canales=canales_1, evocados_co=evocados_co, evocados_in=evocados_in, no_victim=no_victim, 
                           victim=victim, t_init=t_init_1, duration=duration_1, tmin=0.0, tmax=0.796875, pos_legend='lower left')

In [61]:
cluster_1 = ', '.join(canales_1)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1]], 
                         columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_self_vic(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, no_victim=no_victim, victim=victim)
df_medidas.head()

,subject,mean_co_0,mean_in_0,dif_co_in_0
0,21100,-1.003450,1.598310,-2.601759
1,21101,2.061951,1.880054,0.181898
2,21102,-5.217162,-3.681702,-1.535460
3,21103,-1.935621,0.143607,-2.079228
4,21104,0.668344,1.192417,-0.524073


In [62]:
df_medidas['subject']= df_medidas['subject'].astype('int64')
df_medidas.set_index('subject', inplace=True)
medidas = pd.merge(df, df_medidas, left_index=True, right_index=True, how='inner')
medidas.head()

,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,dscore_0,...,TPV,EASPA,ANA,BN,ANC,AGA.1,TAPV,mean_co_0,mean_in_0,dif_co_in_0
subject,,,,,,,,,,,,,,,,,,,,,
21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,-0.197633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.003450,1.598310,-2.601759
21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,-0.116918,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.061951,1.880054,0.181898
21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,0.473119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.217162,-3.681702,-1.535460
21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,0.214867,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.935621,0.143607,-2.079228
21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,0.423545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.668344,1.192417,-0.524073


In [64]:
medidas_clusters_self_vic_350 = medidas[['type','group', 'age', 'scholarship', 'gender', 'laterality', 'dscore_4', 'modo_2', 'mean_co_0', 'mean_in_0',
                                             'dif_co_in_0']]

medidas_clusters_self_vic_350.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 21100 to 24101
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         74 non-null     object 
 1   group        74 non-null     object 
 2   age          74 non-null     int64  
 3   scholarship  74 non-null     int64  
 4   gender       74 non-null     object 
 5   laterality   74 non-null     object 
 6   dscore_4     74 non-null     float64
 7   modo_2       74 non-null     object 
 8   mean_co_0    74 non-null     float64
 9   mean_in_0    74 non-null     float64
 10  dif_co_in_0  74 non-null     float64
dtypes: float64(4), int64(2), object(5)
memory usage: 6.9+ KB


In [65]:
medidas_clusters_self_vic_350.to_csv('medidas_clusters_self_vic_350.csv')

# Análisis por niveles de prejuicio muestra aleatoria

## Creación de diccionarios de ERP por grupos

In [5]:
# Matrices de muestra de sujetos a analizar
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
negative = list(muestra[muestra['prejudice_level']=='negative']['subject'])
neutral = list(muestra[muestra['prejudice_level']=='neutral']['subject'])
positive = list(muestra[muestra['prejudice_level']=='positive']['subject'])

for i in range(len(negative)):
    negative[i] = str(negative[i])
for i in range(len(neutral)):
    neutral[i] = str(neutral[i])
for i in range(len(positive)):
    positive[i] = str(positive[i])

__, __, negative_co, negative_in, __, __, __ = cl.matrices_cluster(negative,evocados_co,evocados_in)
__, __, neutral_co, neutral_in, __, __, __ = cl.matrices_cluster(neutral, evocados_co, evocados_in)
__, __, positive_co, positive_in, tiempos, canales, adjacency = cl.matrices_cluster(positive, evocados_co, evocados_in)

# Matrices de diferencias entre ensayos
negative_dif = negative_co - negative_in
neutral_dif = neutral_co - neutral_in
positive_dif = positive_co - positive_in
print(np.shape(positive_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(19, 205, 64)
(19, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(37, 205, 64)
(37, 205, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(29, 205, 64)
(29, 205, 64)
(29, 205, 64)


## Análisis univariantes masivos

In [7]:
# Análisis diferencias entre ensayos congruentes e incongruentes
X = [negative_dif, neutral_dif, positive_dif]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000190 max=16.702349
Running initial clustering
Using 83 thresholds from 0.20 to 16.60 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [12:15<00:00,    1.36it/s]

Computing cluster p-values


Done.
1290 points selected by TFCE ...


In [8]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura_dif = cl.clusters_color(df=df_p_values)

In [9]:
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>40]

FC4    164.0
AFZ    161.0
AF4    159.0
FPZ    158.0
P9     158.0
CZ     157.0
F4     151.0
AF3    147.0
F2     146.0
F1     146.0
FC2    140.0
FZ     137.0
C2     135.0
FCZ    134.0
CPZ    125.0
C1     125.0
CP1    122.0
CP2    115.0
PO7    108.0
CP4    107.0
FC1     99.0
PZ      94.0
O1      92.0
C3      84.0
FP1     79.0
P1      77.0
F6      69.0
PO3     67.0
IZ      56.0
O2      54.0
TP7     52.0
C4      51.0
P3      49.0
FC3     48.0
CP3     46.0
FC6     45.0
OZ      45.0
dtype: float64

In [94]:
cl.time_intervals(df_electrodos, ['P9', 'PO7', 'O1'])

,cuenta,t_inicial,t_final,delta(ms)
58,25,0.132812,0.226562,93.75


In [77]:
cl.time_intervals(df_electrodos, ['FC4', 'F4', 'AF4', 'F2'])

,cuenta,t_inicial,t_final,delta(ms)
65,31,0.136719,0.253906,117.18750
100,31,0.273438,0.390625,117.18750
164,28,0.535156,0.640625,105.46875


In [88]:
cl.time_intervals(df_electrodos, ['FC4', 'F4', 'AF4', 'AFZ', 'FPZ', 'C2', 'CZ', 'C1', 'CP1'])

,cuenta,t_inicial,t_final,delta(ms)
111,15,0.378906,0.433594,54.6875
164,13,0.593750,0.640625,46.8750


In [40]:
cl.time_intervals(df_electrodos, ['CZ', 'CPZ'])

,cuenta,t_inicial,t_final,delta(ms)
112,12,0.394531,0.437500,42.96875
204,88,0.457031,0.796875,339.84375


In [82]:
cl.time_intervals(df_electrodos, ['FC2', 'FCZ', 'CZ', 'C2', 'FZ', 'F1'])

,cuenta,t_inicial,t_final,delta(ms)
56,12,0.175781,0.218750,42.96875
108,12,0.378906,0.421875,42.96875
204,62,0.558594,0.796875,238.28125


In [97]:
# Análisis ensayos incongruentes
X = [negative_in, neutral_in, positive_in]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000162 max=11.696190
Running initial clustering
Using 58 thresholds from 0.20 to 11.60 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [09:03<00:00,    1.84it/s]

Computing cluster p-values


Done.
67 points selected by TFCE ...


In [98]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura_in = cl.clusters_color(df=df_p_values)

In [99]:
lista, df_electrodos = cl.lista_electrodos(df_p_values=df_p_values,umbral=0.5)
lista[lista>30]

AFZ    191.0
F1     190.0
C3     189.0
FC3    148.0
F3     140.0
CPZ    139.0
FZ     127.0
AF3    125.0
PZ     124.0
FC4    120.0
C1     117.0
CP1    111.0
P1     107.0
FCZ    105.0
FC1     95.0
P3      95.0
FPZ     94.0
AF4     88.0
FC5     83.0
CP2     81.0
CZ      78.0
F2      67.0
C2      62.0
FC2     62.0
FP2     61.0
F4      56.0
CP3     49.0
FC6     45.0
POZ     37.0
F5      37.0
dtype: float64

In [100]:
cl.time_intervals(df_electrodos, ['AFZ', 'AF4', 'FZ'])

,cuenta,t_inicial,t_final,delta(ms)
27,16,0.046875,0.105469,58.59375
63,21,0.167969,0.246094,78.12500
89,13,0.300781,0.347656,46.87500


In [107]:
cl.time_intervals(df_electrodos, ['C3', 'C1', 'CP1'])

,cuenta,t_inicial,t_final,delta(ms)
39,15,0.097656,0.152344,54.68750
72,30,0.167969,0.281250,113.28125
111,33,0.308594,0.433594,125.00000


In [112]:
cl.time_intervals(df_electrodos, ['F1','AFZ'])

,cuenta,t_inicial,t_final,delta(ms)
27,17,0.042969,0.105469,62.50000
125,93,0.128906,0.488281,359.37500
204,76,0.503906,0.796875,292.96875


In [121]:
cl.time_intervals(df_electrodos, ['F1', 'AFZ', 'F3'])

,cuenta,t_inicial,t_final,delta(ms)
125,28,0.382812,0.488281,105.46875
178,40,0.542969,0.695312,152.34375
204,16,0.738281,0.796875,58.59375


In [122]:
# Análisis ensayos incongruentes
X = [negative_co, neutral_co, positive_co]
f_obs, p_values, umbral = cl.analisis_cluster_f(X, adjacency)

stat_fun(H1): min=0.000128 max=8.353431
Running initial clustering
Using 41 thresholds from 0.20 to 8.20 for TFCE computation (h_power=2.00, e_power=0.50)
Found 13120 clusters
Permuting 999 times...


100%|██████████|  : 999/999 [08:45<00:00,    1.90it/s]

Computing cluster p-values


Done.
0 points selected by TFCE ...
No hay clústeres con p-value <= 0.05


In [123]:
df_p_values = cl.p_values_table(p_values, tiempos, canales)
figura_co = cl.clusters_color(df=df_p_values)

## Gráficos de clústeres y medidas de ERP

In [128]:
# Gráfica ERP clúster 1
canales_1 = ['AFZ', 'AF4', 'FZ']
t_init_1 = 0.047
duration_1 = 0.059
fig1 = cl.graficos_niveles(canales=canales_1, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_1, duration=duration_1,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [129]:
# Gráfica ERP clúster 2
canales_2 = ['C3', 'C1', 'CP1']
t_init_2 = 0.098
duration_2 = 0.055
fig2 = cl.graficos_niveles(canales=canales_2, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_2, duration=duration_2,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [130]:
# Gráfica ERP clúster 3
canales_3 = ['P9', 'PO7', 'O1']
t_init_3 = 0.133
duration_3 = 0.094
fig3 = cl.graficos_niveles(canales=canales_3, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_3, duration=duration_3,
                            tmin=0.0, tmax=0.796875, pos_legend='upper left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [133]:
# Gráfica ERP clúster 4
canales_4 = ['AF4', 'F2', 'F4', 'FC4']
t_init_4 = 0.137
duration_4 = 0.117
fig4 = cl.graficos_niveles(canales=canales_4, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_4, duration=duration_4,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [134]:
# Gráfica ERP clúster 5
canales_5 = ['C3', 'C1', 'CP1']
t_init_5 = 0.168
duration_5 = 0.113
fig5 = cl.graficos_niveles(canales=canales_5, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_5, duration=duration_5,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [135]:
# Gráfica ERP clúster 6
canales_6 = ['AF4', 'F2', 'F4', 'FC4']
t_init_6 = 0.273
duration_6 = 0.117
fig6 = cl.graficos_niveles(canales=canales_6, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_6, duration=duration_6,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [136]:
# Gráfica ERP clúster 7
canales_7 = ['C3', 'C1', 'CP1']
t_init_7 = 0.309
duration_7 = 0.125
fig7 = cl.graficos_niveles(canales=canales_7, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_7, duration=duration_7,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [139]:
# Gráfica ERP clúster 8
canales_8 = ['FC4', 'F4', 'AF4', 'AFZ', 'FPZ', 'C2', 'CZ', 'C1', 'CP1']
t_init_8 = 0.379
duration_8 = 0.055
fig8 = cl.graficos_niveles(canales=canales_8, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_8, duration=duration_8,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

In [140]:
# Gráfica ERP clúster 9
canales_9 = ['F1', 'F3', 'FC3', 'AFZ', 'C3']
t_init_9 = 0.383
duration_9 = 0.106
fig9 = cl.graficos_niveles(canales=canales_9, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_9, duration=duration_9,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [141]:
# Gráfica ERP clúster 10
canales_10 = ['CZ', 'CPZ']
t_init_10 = 0.457
duration_10 = 0.340
fig10 = cl.graficos_niveles(canales=canales_10, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_10, duration=duration_10,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [142]:
# Gráfica ERP clúster 11
canales_11 = ['F1', 'AFZ']
t_init_11 = 0.504
duration_11 = 0.293
fig11 = cl.graficos_niveles(canales=canales_11, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_11, duration=duration_11,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [143]:
# Gráfica ERP clúster 12
canales_12 = ['FC2', 'FCZ', 'CZ', 'C2', 'FZ', 'F1', 'CPZ']
t_init_12 = 0.559
duration_12 = 0.238
fig12= cl.graficos_niveles(canales=canales_12, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_12, duration=duration_12,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [144]:
# Gráfica ERP clúster 5 - 7
canales_5_7 = ['C3', 'C1', 'CP1']
t_init_5_7 = 0.168
duration_5_7 = 0.391 - 0.168
fig57 = cl.graficos_niveles(canales=canales_5_7, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_5_7, duration=duration_5_7,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [145]:
# Gráfica ERP clúster 4 - 6
canales_4_6 = ['AF4', 'F2', 'F4', 'FC4']
t_init_4_6 = 0.137
duration_4_6 = 0.391 - 0.137
fig4 = cl.graficos_niveles(canales=canales_4_6, evocados_co=evocados_co, evocados_in=evocados_in, 
                            negative=negative, neutral=neutral, positive=positive, t_init=t_init_4_6, duration=duration_4_6,
                            tmin=0.0, tmax=0.796875, pos_legend='lower left')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [ ]:
cluster_1 = ', '.join(canales_1)
cluster_2 = ', '.join(canales_2)
cluster_3 = ', '.join(canales_3)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1], [cluster_2, t_init_2, t_init_2+duration_2], [cluster_3, t_init_3, t_init_3+duration_3]],
                         columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_grupos(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, exbothsides=exbothsides, exparamilitar=exparamilitar, 
                                 exguerrilla=exguerrilla, victim=victim, control=control)
df_medidas.head()

In [151]:
cluster_1 = ', '.join(canales_1)
cluster_2 = ', '.join(canales_2)
cluster_3 = ', '.join(canales_3)
cluster_4 = ', '.join(canales_4)
cluster_5 = ', '.join(canales_5)
cluster_6 = ', '.join(canales_6)
cluster_7 = ', '.join(canales_7)
cluster_8 = ', '.join(canales_8)
cluster_9 = ', '.join(canales_9)
cluster_10 = ', '.join(canales_10)
cluster_11 = ', '.join(canales_11)
cluster_12 = ', '.join(canales_12)
cluster_4_6 = ', '.join(canales_4_6)
cluster_5_7 = ', '.join(canales_5_7)
clusteres = pd.DataFrame(data=[[cluster_1, t_init_1, t_init_1+duration_1], [cluster_2, t_init_2, t_init_2+duration_2], 
                               [cluster_3, t_init_3, t_init_3+duration_3], [cluster_4, t_init_4, t_init_4+duration_4],
                               [cluster_5, t_init_5, t_init_5+duration_5], [cluster_6, t_init_6, t_init_6+duration_6],
                               [cluster_7, t_init_7, t_init_7+duration_7], [cluster_8, t_init_8, t_init_8+duration_8],
                               [cluster_9, t_init_9, t_init_9+duration_9], [cluster_10, t_init_10, t_init_10+duration_10],
                               [cluster_11, t_init_11, t_init_11+duration_11], [cluster_12, t_init_12, t_init_12+duration_12],
                               [cluster_4_6, t_init_4_6, t_init_4_6+duration_4_6], [cluster_5_7, t_init_5_7, t_init_5_7+duration_5_7]],
                               columns=['electrodos','t_inicial (s)','t_final (s)'])
df_medidas= cl.batch_medidas_niveles(clusteres=clusteres, evocados_co=evocados_co, evocados_in=evocados_in, 
                                     positive=positive, negative=negative, neutral=neutral)
df_medidas.head()

c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\funciones_clusters.py:204: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (0.796875 sec)
  evoked_ROI.crop(tmin = tmin, tmax = tmax)
c:\Users\jhquiza\OneDrive - Universidad de M

,subject,mean_co_0,mean_in_0,dif_co_in_0,mean_co_1,mean_in_1,dif_co_in_1,mean_co_2,mean_in_2,dif_co_in_2,...,dif_co_in_10,mean_co_11,mean_in_11,dif_co_in_11,mean_co_12,mean_in_12,dif_co_in_12,mean_co_13,mean_in_13,dif_co_in_13
0,23009,0.332622,-0.438057,0.770679,0.295931,0.159753,0.136179,0.724781,-0.822040,1.546821,...,0.328664,-0.101525,-0.229398,0.127874,0.570865,0.734140,-0.163275,0.667198,0.280645,0.386553
1,22111,1.146591,0.193997,0.952594,-0.377825,0.129504,-0.507328,-2.326587,-1.158542,-1.168045,...,0.586332,0.807752,0.658902,0.148850,2.236373,1.360146,0.876226,1.547450,1.378624,0.168826
2,23013,-0.114438,-0.690528,0.576091,0.542943,-0.230955,0.773897,-0.744117,0.089469,-0.833586,...,0.343149,0.197375,0.708460,-0.511085,1.183873,0.113425,1.070448,0.675106,0.578845,0.096262
3,21122,-0.021984,0.293676,-0.315660,-0.104398,0.035492,-0.139889,-0.656629,-1.216555,0.559926,...,-0.821218,-1.532599,-0.883218,-0.649382,-1.617307,0.109678,-1.726985,0.655975,-0.129273,0.785248
4,22100,0.946447,-0.747086,1.693533,-0.898980,-0.852293,-0.046687,-0.572723,-0.606529,0.033807,...,0.104485,-0.749376,-0.650888,-0.098489,0.933729,0.418694,0.515035,-0.248561,-1.025580,0.777019


In [153]:
df_medidas['subject']= df_medidas['subject'].astype('int64')
medidas = pd.merge(muestra, df_medidas, left_on='subject', right_on='subject', how='inner')
medidas = medidas.sort_values(by=['subject'])
medidas.head()

,subject,type,exposure_level,group,victim_self_id,dscore_4,prejudice_level,mean_co_0,mean_in_0,dif_co_in_0,...,dif_co_in_10,mean_co_11,mean_in_11,dif_co_in_11,mean_co_12,mean_in_12,dif_co_in_12,mean_co_13,mean_in_13,dif_co_in_13
0,24013,victim,high,victim,yes,-0.039706,neutral,0.133753,-0.497080,0.630833,...,0.716602,1.516743,1.084770,0.431973,0.676385,-0.252561,0.928946,-0.747492,-0.785411,0.037919
1,23004,excombatant,high,exparamilitar,yes,0.555094,positive,0.557428,-0.200206,0.757634,...,-0.655036,0.048521,0.219497,-0.170976,-0.096163,0.118143,-0.214306,0.566458,-0.086958,0.653416
2,24065,victim,high,victim,yes,0.236350,positive,-0.275010,0.986416,-1.261426,...,-1.311387,0.190117,1.640601,-1.450485,0.821194,2.661289,-1.840095,-0.767722,0.081816,-0.849538
3,23009,excombatant,high,exparamilitar,yes,-0.439115,negative,0.332622,-0.438057,0.770679,...,0.328664,-0.101525,-0.229398,0.127874,0.570865,0.734140,-0.163275,0.667198,0.280645,0.386553
4,21134,excombatant,high,exparamilitar,yes,-0.099490,neutral,0.044857,-0.078540,0.123397,...,2.755989,2.710016,0.018415,2.691601,2.334177,1.264382,1.069795,0.076576,-0.599650,0.676226


In [155]:
medidas.to_csv('medidas_clusters_niveles.csv', index=False)

## Extracción muestra aleatoria

In [ ]:
# Discretización de la variable IAT score
from sklearn.preprocessing import KBinsDiscretizer
df = pd.read_csv('Datos_IAT_comportamentales_personales.csv')
df.set_index('subject', inplace=True)

# Se eliminan sujetos del grupo ambos bandos
df = df[df['group']!='ex-both-sides']

dscore_4 = df['dscore_4'].copy()
dscore_4.dropna(inplace=True)
disc = KBinsDiscretizer(n_bins=3, encode='ordinal',strategy='kmeans').fit(np.array(dscore_4).reshape(-1,1))
print('límites grupos: ',disc.bin_edges_)
y_discretized = KBinsDiscretizer(n_bins=3, encode='ordinal',strategy='kmeans').fit_transform(np.array(dscore_4).reshape(-1,1))
labels = pd.DataFrame(y_discretized, columns=['label'], index=dscore_4.index)
labels = labels.astype('object')
labels.groupby('label').size()

In [ ]:
# Etiquetado de niveles de prejuicio
df = df[['type','exposure_level','group','victim_self_id','dscore_4' ]].copy()
df.dropna(inplace=True)
salidas = pd.merge(df, labels, left_index=True, right_index=True)
salidas.rename({'label':'prejudice_level_ind'}, axis=1, inplace=True)
salidas.reset_index(inplace=True)
salidas['prejudice_level']='a'
for i in range(len(salidas)):
    if salidas['prejudice_level_ind'][i]==2.0:
        salidas['prejudice_level'].at[i]='positive'
    elif salidas['prejudice_level_ind'][i]==1.0:
        salidas['prejudice_level'].at[i]='neutral'
    else:
        salidas['prejudice_level'].at[i]='negative'

In [ ]:
# Selección de muestra aleatoria
from sklearn.model_selection import train_test_split
X = salidas[['subject','type','exposure_level','group','victim_self_id','dscore_4' ]]
y = salidas['prejudice_level']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.06, stratify=y, random_state=72)
salidas_aleatorias = pd.merge(X_train, y_train, left_index=True, right_index=True)
salidas_aleatorias.set_index('subject', inplace=True)
salidas_aleatorias.head(10)
#salidas_aleatorias.to_csv('muestra_aleatoria_sin_ambos.csv', index=True)